<a href="https://colab.research.google.com/github/shahriarivari/Persian_sentiment_analysis/blob/main/Train_BERT_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#some pip installs

In [1]:
!pip install tokenizers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00


#Loading the dataset
We are using Huggingface's data hub

In [2]:
# some pip installs
import os
from tokenizers import BertWordPieceTokenizer
from datasets import load_dataset, Dataset

##Loading the data

In [3]:
# You should just change this part in order to download your
# parts of corpus.
indices = {
    "train": [81, 14, 3,
              # 94, 35,
              # 41, 28, 67, 55, 79
              ],
    "test": [0,
            #  1
             ]
}

N_FILES = {
    "train": 126,
    "test": 3
}
_BASE_URL = "https://huggingface.co/datasets/SLPL/naab/resolve/main/data/"
data_url = {
    "train": [_BASE_URL + "train-{:05d}-of-{:05d}.txt".format(x, N_FILES["train"]) for x in range(N_FILES["train"])],
    "test": [_BASE_URL + "test-{:05d}-of-{:05d}.txt".format(x, N_FILES["test"]) for x in range(N_FILES["test"])],
}
for index in indices['train']:
    assert index < N_FILES['train']
for index in indices['test']:
    assert index < N_FILES['test']
data_files = {
    "train": [data_url['train'][i] for i in indices['train']],
    "test": [data_url['test'][i] for i in indices['test']]
}


In [4]:
dataset_train = load_dataset('text', data_files=data_files["train"], use_auth_token=False)
dataset_test = load_dataset('text', data_files=data_files["test"], use_auth_token=False)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2483: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2483: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

##Turn each dataset into a .txt file to feed to the tokenizer

In [7]:
for index, dataset in enumerate([dataset_train, dataset_test]):
  current_dataset = dataset['train']['text']
  # Save the current slice to a file
  current_file = f'dataset{index + 1}.txt'
  print(f"starting writing dataset{index + 1}")
  with open(current_file, 'w', encoding='utf-8') as file:
      for text in current_dataset:
          file.write(text + '\n')

  print(f"finished writing dataset{index + 1}")
  # Free up memory by deleting the variable
  del current_dataset

starting writing dataset1
finished writing dataset1
starting writing dataset2
finished writing dataset2


In [8]:
# Set your paths and file names
tokenizer_output_dir = "bert_tokenizer"
if not os.path.isdir(tokenizer_output_dir):
    os.mkdir(tokenizer_output_dir)

##Training the tokenizer

In [9]:
# Training a WordPiece Tokenizer
files = ["dataset1.txt", "dataset2.txt"]
# Parameters for Tokenizer Training
vocab_size = 10_000
min_frequency = 100
special_tokens = ["[PAD]", "[MASK]", "[CLS]", "[SEP]", "[UNK]"]

# Initialize the WordPiece tokenizer for BERT
tokenizer = BertWordPieceTokenizer()
print("starting to train the toeknizer")

# Train the tokenizer
tokenizer.train(
    files=files,
    vocab_size=vocab_size,
    min_frequency=min_frequency,
    show_progress=True,
    special_tokens=special_tokens,
)
print("training is done now saving...")
# Save the trained tokenizer
tokenizer.save_model(tokenizer_output_dir)

starting to train the toeknizer
training is done now saving...


['bert_tokenizer/vocab.txt']